In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pyspark

import datetime as dt

import tabula
import joblib

In [11]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 60)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import sklearn
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate \
                                    ,cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,recall_score,precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.metrics import roc_curve, auc

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE

In [28]:
# Create passengers df
passengers = tabula.read_pdf('./data/raw/passengers/usa2017enplanements.pdf',multiple_tables=True, pages='all', guess=False, lattice=True)
df = pd.concat(passengers, axis=0, ignore_index=True)
df = df[[3, 8, 9, 10]]
df = df[1:]
col = {3: 'airport_code', 8: 'cy17', 9: 'cy16', 10:'perc_change'}
df.rename(columns=col, inplace=True)

df = df[~df.airport_code.isin(['Locid'])]
df.dropna(inplace=True)

df.cy17 = df.cy17.str.replace(',','')
df.cy16 = df.cy16.str.replace(',','')
df.cy17 = df.cy17.astype('int64')
df.cy16 = df.cy16.astype('int64')

df.perc_change = df.perc_change.str.replace('%','')
df.perc_change = df.perc_change.astype('float64')
df.reset_index(drop=True, inplace=True)

In [29]:
# df.to_csv('./data/clean/usa2016-17-enplanements.csv')
# df.to_pickle('./data/clean/usa2016-17-enplanements.pkl')
df= pd.read_pickle('./data/clean/usa2016-17-enplanements.pkl')

In [50]:
df.cy17.iloc[0] / df.cy17.sum()

0.05868591831574753

In [57]:
df.cy17.iloc[:40].sum() / df.cy17.sum()

0.797069865424873

In [14]:
df[df.airport == 'RSW']

,airport,cy17,cy16,perc_change
47,RSW,4364217,4239261,2.950


In [2]:
!ls models

bernoulliNB_12.04.2018.joblib
catboost_12.05.2018.joblib
catboost_new_12.05.2018.joblib
catboost_search_12.05.2018.joblib
extratrees_12.04.2018.joblib
gradboost_12.04.2018.joblib
logit_12.04.2018.joblib
multinomialNB_12.04.2018.joblib
preliminary
rf_12.04_feature_importance_list
rf_months_12.03.2018
rf_months_12.03.2018.joblib
rf_subset_airports_airlines_12.04.2018.joblib
supportvector_12.04.2018.joblib
transformers
xgb_12.05.2018.joblib


In [6]:
import joblib

In [14]:
from sklearn.externals import joblib

In [15]:
model = joblib.load(open('./models/catboost_new_12.05.2018.joblib', 'rb'))

In [16]:

example = {
  'airport_code': 'SEA',  # str
  'airline': 'Emirates',    # str
  'claim_type': 'PropertyLoss',    # str
  'claim_site': 'Checkpoint',  # str
  'item_category': 'Clothing',  # str
  'num_items_or_incidents_claimed': 1,  # int
  'days_waited_to_file_claim': 7,  # int
  'Month_received': '1'  # int (1-12)
}


def make_prediction(features):
    '''
    :param features: dictionary like 'example' above
    :return: 2 pair dict of binary outcome (compensate and not compensate) and the probablity
    '''
    X = pd.DataFrame(data=features, index=[0])

    categorical = ['airport_code', 'airline', 'claim_type', 'claim_site', 'Month_received']
    continuous = ['days_waited_to_file_claim', 'num_items_or_incidents_claimed']

    trans_dir = './stat_models/transformers'
    enc = joblib.load(f'{trans_dir}/onehotencode.joblib')
    onehotarray = enc.transform(X[categorical])

    ss = joblib.load(f'{trans_dir}/standardscaler.joblib')
    continuousarray = ss.transform(X[continuous])

    mlb = joblib.load(f'{trans_dir}/item_category.joblib')
    onehot_itemcategories = mlb.transform(X['item_category'].str.replace(' ', '').str.split(pat=';'))

    X = np.concatenate((onehotarray, continuousarray, onehot_itemcategories), axis=1)

    prob_receive_compensation = model.predict_proba(X)[0, 1]

    result = {
        'compensation': int(prob_receive_compensation > 0.5),
        'prob_receive_compensation': prob_receive_compensation
    }
    return result

In [17]:
make_prediction(example)

FileNotFoundError: [Errno 2] No such file or directory: './stat_models/transformers/onehotencode.joblib'